In [2]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [6]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [7]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NSW"]
data_file = 'gru_inputs_simple_v8_kitchen_sink.fth'
# data_file = r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_simple_TESTING_v6.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True, date_filter=date)

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
(1253403, 23)
28
Latest date = 2023-05-26 00:00:00
size after state filter (311201, 25)
(311201, 25)
Latest date = 2023-05-26 00:00:00
num_features_per_dog=28


  0%|          | 0/6959 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/11199 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
 44%|████▍     | 4944/11199 [01:26<00:17, 358.86it/s]

index 7 is out of bounds for dimension 0 with size 5


 68%|██████▊   | 7565/11199 [02:04<00:12, 300.75it/s]

index 7 is out of bounds for dimension 0 with size 7


 84%|████████▍ | 9407/11199 [02:14<00:05, 324.58it/s]

index 5 is out of bounds for dimension 0 with size 3


100%|██████████| 11199/11199 [02:20<00:00, 79.92it/s] 


number of races = 11199, number of unique dogs = 6960


In [8]:
races = raceDB.get_test_input([1,2,3])
races[0].race_dist

tensor([300.], device='cuda:0', dtype=torch.float64)

In [9]:
races[0].race_dist

tensor([300.], device='cuda:0', dtype=torch.float64)

In [10]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 0, Test examples 11199


In [11]:
raceDB.create_dogs_test_split_date()

100%|██████████| 6959/6959 [00:00<00:00, 133705.42it/s]


In [12]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 6960/6960 [00:01<00:00, 6553.88it/s]


In [13]:
all_dog_input = [list(d.races.values()) for d in raceDB.dogsDict.values() if d.races.values()]

In [14]:
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in all_dog_input]], enforce_sorted=False) 

In [15]:
packed_y = packed_y.to('cuda:0')

In [16]:
# model_name = 'curious-durian-408'
# model,hidden_state_dict = setup_model(model_name)

In [17]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [18]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [0, 0, 0]


In [19]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    # print(bi)
    # print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in tqdm(raceDB.dog_ids):
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in batch if x in dog.races.keys()]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6959 [00:00<?, ?it/s]

  0%|          | 0/6959 [00:00<?, ?it/s]

  0%|          | 0/6959 [00:00<?, ?it/s]

In [20]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [21]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [0, 0, 0]
Train examples [0, 0, 0]
Train examples [0, 0, 0]
Train examples [0, 0, 0]


In [22]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = ""#[pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_x_basic = ""#[pack_sequence([torch.stack(n,0) for n in [[z.stats for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = ""#pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
packed_y_basic = pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [23]:
raceDB.batches = {'num_batches':len(train_dogs),
                  'dogs':train_dogs,
                  'train_dog_input':train_dog_input,
                  'batch_races':batch_races,
                  'batch_races_ids':batch_races_ids,
                  'packed_x':packed_x,
                  'packed_x_basic':packed_x_basic,
                  'packed_y_basic':packed_y_basic,
                    'packed_y':packed_y}

In [24]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU - updates - shuffle", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)
      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        input_size = raceDB.batches['packed_y'][0].data[0].shape[0]

      print(f"{input_size=}")

      # raceDB.batches['packed_y'][0] = raceDB.batches['packed_y'][0]._replace(data=)

      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_BN(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      criterion = nn.CrossEntropyLoss(reduction='none')
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      model = model.to(device)

      print(model)

      # training_testing_gru.validate_model_v3(model,raceDB, criterion,epoch=0)

      model.eval()
      training_testing_gru.validate_model_v3(model,raceDB, criterion,epoch=0)
      # print(ahhh)

      # # and use them to train the model
      # try:
      #   # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
      #   training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config, update=True)
      # except (KeyboardInterrupt) as error:
      #   print(error)
      #   print("finished Early")

      # raceDB.create_hidden_states_dict_v2()
      # model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [25]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                       'datafile':data_file,
                       'stats':raceDB.stats_cols,
                       'input_type':'basic',
                       'races':states,
                       'latest_date':raceDB.latest_date,
                       'num_layers':2,
                       'batch_size': 750,
                       'dropout': 0.3,
                       'epochs': 10_000,
                       'learning_rate': 0.0003,
                       'optimizer': 'adamW',
                       'f0_layer_size':128,
                       'f1_layer_size':64}

In [26]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [27]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [28]:
torch.cuda.empty_cache()

In [29]:
original = packed_y_basic

In [30]:
original_data = packed_y_basic.data

In [31]:
x = original_data.to('cpu')
x

tensor([[ 5.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  1.4000e-01,
          5.1000e+01,  1.9608e-02],
        [ 8.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+00,
          3.4000e+00,  2.9412e-01],
        [ 3.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  2.4000e-01,
          5.0000e+00,  2.0000e-01],
        ...,
        [ 6.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  3.1000e-01,
          2.1000e+01,  4.7619e-02],
        [ 8.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  4.7000e-01,
          1.0100e+02,  9.9010e-03],
        [ 7.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.8000e-01,
          2.3000e+01,  4.3478e-02]], dtype=torch.float64)

In [32]:
torch.max(x, dim=0)

torch.return_types.max(
values=tensor([8.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 4.3200e+01, 8.0000e+00,
        7.6000e+02, 8.0000e+00, 1.8859e+01, 2.8107e+04, 1.6338e+03, 1.4050e+01,
        1.0225e+02, 3.9560e+01, 4.1024e+01, 2.5327e+01, 1.0000e+00, 9.0000e+00,
        7.0000e+00, 2.2700e+02, 1.0000e+00, 1.0000e+00, 4.3200e+01, 4.9210e+01,
        2.9600e+01, 1.4050e+01, 4.1000e+02, 1.0000e+00], dtype=torch.float64),
indices=tensor([    1,     3,     0,     1,  4561,     2, 52454,     0, 40460, 59173,
         2138, 45881, 10334,  4069, 45881, 13187,     7, 67587,   475, 80181,
          191,    16, 11520, 32301, 45881, 45881, 11545,   693]))

In [33]:
idx = torch.randperm(original_data.shape[0])
original_data[idx].view(original_data.size())

tensor([[ 8.0000,  0.0000,  0.0000,  ...,  0.3500,  3.4000,  0.2941],
        [ 1.0000,  1.0000,  0.0000,  ...,  0.1600, 26.0000,  0.0385],
        [ 1.0000,  1.0000,  0.0000,  ...,  0.0600,  7.0000,  0.1429],
        ...,
        [ 2.0000,  1.0000,  0.0000,  ..., -1.0000,  3.6000,  0.2778],
        [ 2.0000,  1.0000,  0.0000,  ..., -1.0000,  2.3000,  0.4348],
        [ 5.0000,  0.0000,  1.0000,  ..., -1.0000,  5.5000,  0.1818]],
       device='cuda:0', dtype=torch.float64)

In [34]:
x = x[1:100]
x

tensor([[ 8.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+00,
          3.4000e+00,  2.9412e-01],
        [ 3.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  2.4000e-01,
          5.0000e+00,  2.0000e-01],
        [ 1.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.5000e-01,
          1.3000e+01,  7.6923e-02],
        ...,
        [ 1.0000e+00,  1.0000e+00,  0.0000e+00,  ..., -1.0000e+00,
          9.0000e+00,  1.1111e-01],
        [ 2.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  2.1000e-01,
          8.0000e+00,  1.2500e-01],
        [ 2.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  3.1000e-01,
          7.1000e+01,  1.4085e-02]], dtype=torch.float64)

In [35]:
torch.max(x,dim=0)

torch.return_types.max(
values=tensor([8.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 3.6100e+01, 8.0000e+00,
        6.2200e+02, 8.0000e+00, 1.7802e+01, 6.4063e+03, 1.8110e+01, 5.2000e-01,
        1.9000e+01, 2.6990e+01, 1.7872e+01, 1.5258e+01, 1.0000e+00, 8.0000e+00,
        5.0000e+00, 1.7900e+02, 5.0000e-01, 1.0000e+00, 3.5500e+01, 3.6630e+01,
        1.6040e+01, 5.2000e-01, 1.0100e+02, 5.4054e-01], dtype=torch.float64),
indices=tensor([ 0,  2,  1,  0, 82,  1, 95,  4, 32, 45, 65, 89, 72, 72, 32, 19,  6, 17,
        95, 11, 40, 15, 82, 95, 92, 89,  5, 15]))

In [36]:
idx = torch.randperm(x.size(0))
i= 5
y = x
y[:, i] = x[idx,i] 
print(y[:, i])

tensor([5., 8., 8., 6., 7., 8., 5., 5., 8., 7., 8., 5., 5., 5., 5., 8., 3., 6.,
        7., 7., 5., 5., 5., 8., 8., 7., 8., 7., 8., 5., 7., 6., 6., 7., 8., 7.,
        7., 8., 7., 5., 8., 8., 4., 7., 8., 7., 6., 5., 7., 7., 8., 7., 8., 8.,
        8., 5., 5., 8., 5., 8., 5., 5., 5., 6., 5., 8., 5., 7., 6., 3., 8., 8.,
        8., 5., 8., 8., 4., 5., 5., 5., 5., 8., 5., 1., 5., 5., 8., 5., 8., 8.,
        8., 6., 7., 8., 8., 7., 8., 8., 4.], dtype=torch.float64)


In [37]:
x.size(1)

28

In [38]:
original_data

tensor([[ 5.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  1.4000e-01,
          5.1000e+01,  1.9608e-02],
        [ 8.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+00,
          3.4000e+00,  2.9412e-01],
        [ 3.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  2.4000e-01,
          5.0000e+00,  2.0000e-01],
        ...,
        [ 6.0000e+00,  0.0000e+00,  1.0000e+00,  ...,  3.1000e-01,
          2.1000e+01,  4.7619e-02],
        [ 8.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  4.7000e-01,
          1.0100e+02,  9.9010e-03],
        [ 7.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.8000e-01,
          2.3000e+01,  4.3478e-02]], device='cuda:0', dtype=torch.float64)

In [41]:
wandb_config_static['variable'] = 'boxpos'
raceDB.batches['packed_y_basic'] = original_pack
x = original_pack.data.to('cpu')
idx = torch.randperm(x.size(0))
x[:, 1:3] = x[idx,1:3] 
raceDB.batches['packed_y_basic'] = raceDB.batches['packed_y_basic']._replace(data=x.to('cuda:0'))
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='glamorous-lion-731', prev_model_version='400')

{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-05-26T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0003, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'variable': 'boxpos'}
10000
{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_

100%|██████████| 6960/6960 [00:00<00:00, 365970.34it/s]


filled =3856
empty  =3104
0.5540229885057472null_dog=0
GRUNetv3_BN(
  (gru): GRU(28, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
epoch,▁▁
flat_simple,▁▁
loss_val,▁▁
multibet outlay < 30,▁▁
multibet profit,▁▁
multibet profit < 30,▁▁
multibet profit < 30 sd,▁▁


In [39]:
original_pack = pack_sequence([torch.stack(n,0) for n in [[z.stats.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)
stat_cols= ['box','inside','midfield','wide','weight','DogGrade','dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']
for i in range(4,len(stat_cols)):
    wandb_config_static['variable'] = stat_cols[i]
    raceDB.batches['packed_y_basic'] = original_pack
    x = original_pack.data.to('cpu')
    idx = torch.randperm(x.size(0))
    x[:, i] = x[idx,i] 
    raceDB.batches['packed_y_basic'] = raceDB.batches['packed_y_basic']._replace(data=x.to('cuda:0'))
    (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='glamorous-lion-731', prev_model_version='400')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-05-26T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0003, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'variable': 'weight'}
10000
{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_

100%|██████████| 6960/6960 [00:00<00:00, 434604.08it/s]


filled =3856
empty  =3104
0.5540229885057472null_dog=0
GRUNetv3_BN(
  (gru): GRU(28, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
epoch,▁▁
flat_simple,▁▁
loss_val,▁▁
multibet outlay < 30,▁▁
multibet profit,▁▁
multibet profit < 30,▁▁
multibet profit < 30 sd,▁▁


...

ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
epoch,▁▁
flat_simple,▁▁
loss_val,▁▁
multibet outlay < 30,▁▁
multibet profit,▁▁
multibet profit < 30,▁▁
multibet profit < 30 sd,▁▁


{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-05-26T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0003, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'variable': 'last_start_prob'}
10000
{'hidden_size': 128, 'datafile': 'gru_inputs_simple_v8_kitchen_sink.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_ho

100%|██████████| 6960/6960 [00:00<00:00, 463591.49it/s]


filled =3856
empty  =3104
0.5540229885057472null_dog=0
GRUNetv3_BN(
  (gru): GRU(28, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
epoch,▁▁
flat_simple,▁▁
loss_val,▁▁
multibet outlay < 30,▁▁
multibet profit,▁▁
multibet profit < 30,▁▁
multibet profit < 30 sd,▁▁


In [28]:
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file='peachy-darkness-493', prev_model_version='1600')
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='bright-bee-555', prev_model_version='1600')
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='curious-sweep-4', prev_model_version='900')
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='visionary-valley-541', prev_model_version='3500')
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='kind-wood-610', prev_model_version='1300')
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='jolly-sweep-7', prev_model_version='900')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 128, 'datafile': 'gru_inputs_new_simple_test.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-05-02T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 0.0003, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64}
10000
{'hidden_size': 128, 'datafile': 'gru_inputs_new_simple_test.fth', 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']]", 'input_type': 'basic', 'races': ['NSW'], 'latest_date': '2023-05-02T00:00:00', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'ep

 55%|█████▌    | 3670/6671 [00:00<00:00, 36666.55it/s]

Empty: 645474873 JONAH LOVE
Empty: 651385170 EXTREME ROCK
Empty: 583222624 MORAINE JADE
Empty: 504049596 MAELYNN BALE
Empty: 549963741 SWEET PANAMA
Empty: 705701202 TINY PSYCHO
Empty: 603192479 ELITE BULLET
Empty: 654230427 BOOK BOOK
Empty: 638802618 LUCKY AS BEN
Empty: 638803146 PHANTOM ARIZONA
Empty: 638802698 CHICAGO REIGN
Empty: 638803119 BOLAN ROAD
Empty: 670507887 WILD STREAK
Empty: 654231426 MAGIC SECRET
Empty: 718777802 ALINTA FIRE
Empty: 669960260 KRYPTO BOY
Empty: 677833466 EYES ON GOLD
Empty: 654230432 LUCKY BUNDY
Empty: 634935392 MISS SNAP
Empty: 677834000 MAGIC TALENT
Empty: 677833465 WILLINA CHARM
Empty: 654230904 BUNDY ROSE
Empty: 572578374 HYLAND HAKUNA
Empty: 629791611 SILLY SAUSAGE
Empty: 638802753 BUSTER SLOY
Empty: 629792190 ZAMBORA BLASTER
Empty: 576142515 THREE'S A CHARM
Empty: 520643353 FRIEND OR FOE
Empty: 341855900 DEB'S BLUE BOY
Empty: 638803628 GREEK FLAME
Empty: 654230923 SNIPER'S KING
Empty: 617355249 I'M FULL ON
Empty: 609359802 ZIPPING CUTIE
Empty: 576143

100%|██████████| 6671/6671 [00:00<00:00, 15079.06it/s]


Empty: 747910335 DANCING FLAMINGO
Empty: 718777214 REDSHIFT PAPA
Empty: 870438061 ORSON ARIA
Empty: 765532187 CLANCY'S TAVERN
Empty: 714917642 NOBBY'S GIRL
Empty: 710477645 WENDY ROCKS
Empty: 623992507 ROCHILL PEARL
Empty: 702436600 LIMBUNYA
Empty: 740359816 HAYMAN BEACH
Empty: 695963210 SET TO RUN
Empty: 771724529 RITZA LEWIS
Empty: 650078173 AYLA
Empty: 734719590 GAIL UNLEASHED
Empty: 733458991 AKELA ZORO
Empty: 775589228 ROMAN WOLF
Empty: 747910340 DAPPER FOX
Empty: 747910337 DAPPER RICH
Empty: 669959782 NEW HIGHS
Empty: 606761154 VENOM SOLDIER
Empty: 718777964 TAW COMMANDER
Empty: 710477632 BE DIPLOMATIC
Empty: 632134757 ASTON DOUGIE
Empty: 632134756 ASTON PROSPECT
Empty: 716345354 TIGGERLONG EGG
Empty: 615180911 ASTON HOGAN
Empty: 752070660 PREMIER LOU
Empty: 700566858 MS. BLUECHIP
Empty: 718777338 FLASH TIGER
Empty: 740359273 FRANKY BEAR
Empty: 718777944 SUSIE'S SO FAST
Empty: 718776823 MR. ARIA
Empty: 718777324 JOHNNY FOR SURE
Empty: 877724787 JACK NELSON
Empty: 623993510 CHAIR

GRUNetv3_LN(
  (gru): GRU(20, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)
created path


ROI < 30,▁
ROI < 30 2,▁
accuracy,▁
epoch,▁
flat_simple,▁
loss_val,▁
multibet outlay < 30,▁
multibet profit,▁
multibet profit < 30,▁
multibet profit < 30 sd,▁
profit_relu,▁


In [29]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [30]:
race = raceDB.batches['batch_races'][0][2]
[d for d in race.dogs]

IndexError: list index out of range